# Import Library

In [1]:
from __future__ import print_function
from imutils.video import VideoStream
import cv2
import numpy as np
from adafruit_servokit import ServoKit
import time
import os
import RPi.GPIO as GPIO
import imutils

# Set up servo & led

In [2]:
GPIO.cleanup()

In [3]:
kit=ServoKit(channels=16)
# LED initial
redLed = 17
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(redLed, GPIO.OUT)
# close led
GPIO.output(redLed, GPIO.LOW)
ledOn = False

In [4]:
pan =  90
tilt = 90
# set servo initial
kit.servo[0].angle=pan
kit.servo[1].angle=tilt

# Threading operation management & Debug threading workwell

In [5]:
import threading 
import ctypes
import inspect

# set the end code
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# Set up CSI Camera

In [6]:
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

frame_img = widgets.Image(format='jpeg', width=320, height=240)
display(frame_img)

Image(value=b'', format='jpeg', height='240', width='320')

In [7]:
dispW=320
dispH=240
flip=4
camSet='nvarguscamerasrc !  video/x-raw(memory:NVMM), width=3264, height=2464, format=NV12, framerate=21/1 ! nvvidconv flip-method='+str(flip)+' ! video/x-raw, width='+str(dispW)+', height='+str(dispH)+', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink'
cam= cv2.VideoCapture(camSet)

width=cam.get(cv2.CAP_PROP_FRAME_WIDTH)
height=cam.get(cv2.CAP_PROP_FRAME_HEIGHT)
print('width:',width,'height:',height)

width: 320.0 height: 240.0


# Import face & eyes detect model

In [8]:
face_cascade = cv2.CascadeClassifier('./images/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./images/haarcascade_eye.xml')

# Set up face tracking code & Tunning PID

In [9]:
def Video_display():
    global pan
    global tilt
    global ledOn
    while True: 
        ret,frame = cam.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.3,5)

        for(x,y,w,h) in faces:
            # draw face rectangular
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            # middle of face rectangular
            Xcent = x + w/2
            Ycent = y + h/2
            errorPan  = Xcent - dispW/2  # if face middle at gragh right side => errorPan = positive, otherwise negative
            errorTilt = Ycent - dispH/2  # if face middle at gragh down side => errorPan = positive, otherwise negative
            # if errorPan > 15 pixel the servo will be adjust, limit frequency to adjust servo which make servo burn
            if abs(errorPan)>15:
                pan=pan-errorPan/50 # 50 is hyperparameter, if the value is big => servo move fast, otherwise slow
            if abs(errorTilt)>15:
                tilt=tilt-errorTilt/50
            # set the servo at 0~180
            if pan > 180:
                pan = 180
                print("Pan out of Range")
            if pan < 0:
                pan = 0
                print("pan Out of Range")
            if tilt > 180:
                tilt = 180
                print("Pan out of Range")
            if tilt < 0:
                tilt = 0
                print("pan Out of Range")
            # adjust servo
            kit.servo[0].angle=180-pan
            kit.servo[1].angle=tilt
            
            # detect eyes
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]        
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex,ey,ew,eh) in eyes:
                cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2) 
                
            # when eye detect => led on
            if len(eyes) > 0:
                if not ledOn:
                    GPIO.output(redLed, GPIO.HIGH)
                    ledOn = True
            #else:
            elif ledOn:
                GPIO.output(redLed, GPIO.LOW)
                ledOn = False
        frame_img.value = bgr8_to_jpeg(frame)
    cap.release()

# Start the code

In [10]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

# Terminate program & Release resource

In [11]:
# quit program
stop_thread(t)
GPIO.cleanup()